<a href="https://colab.research.google.com/github/ajeancharles/DCGAN_Exploration/blob/main/text_generation_aesop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
url = r'https://www.gutenberg.org/cache/epub/11339/pg11339.txt'

In [2]:
import gdown
import os

In [3]:
os.getcwd()

'/content'

In [4]:
os.listdir((os.getcwd()))

['.config', 'aesop', 'sample_data']

In [5]:
# Root directory for the dataset
data_root = '/content/aesop'


# Path to download the dataset to
download_path = f'{data_root}/fables.txt'

# Create required directories 
if not os.path.exists(data_root):
  os.makedirs(data_root)

# Download the dataset from google drive
gdown.download(url, download_path, quiet=False)


Downloading...
From: https://www.gutenberg.org/cache/epub/11339/pg11339.txt
To: /content/aesop/fables.txt
243kB [00:00, 1.43MB/s]


'/content/aesop/fables.txt'

In [6]:
import re
from keras.preprocessing.text import Tokenizer

filename = "/content/aesop/fables.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read()

seq_length = 20
start_story = '| ' * seq_length

# CLEANUP
text = text.lower()
text = start_story + text
text = text.replace('\n\n\n\n\n', start_story)
text = text.replace('\n', ' ')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)


# TOKENIZATION
tokenizer = Tokenizer(char_level = False, filters = '')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0]

In [7]:
import numpy as np
from keras.utils import np_utils

def generate_sequences(token_list, step):

    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])

    y = np_utils.to_categorical(y, num_classes = total_words)

    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")

    return X, y, num_seq

step = 1
seq_length = 20
X, y, num_seq = generate_sequences(token_list, step)

X = np.array(X)
y = np.array(y)

Number of sequences: 58912 



In [ ]:
from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop

n_units = 256
embedding_size = 100

text_in = Input(shape = (None,))
x = Embedding(total_words, embedding_size)(text_in)
x = LSTM(n_units)(x)
x = Dropout(0.2)(x)
text_out = Dense(total_words, activation = 'softmax')(x)

model = Model(text_in, text_out)

opti = RMSprop(learning_rate = 0.001)
model.compile(loss='categorical_crossentropy', optimizer=opti)

epochs = 100
batch_size = 32
model.fit(X, y, epochs=epochs, batch_size=batch_size, shuffle = True)

Epoch 1/100
1841/1841 [==============================] - 11s 5ms/step - loss: 5.2190
Epoch 2/100
1841/1841 [==============================] - 8s 5ms/step - loss: 4.8269
Epoch 3/100
1841/1841 [==============================] - 8s 5ms/step - loss: 4.6783
Epoch 4/100
1841/1841 [==============================] - 8s 4ms/step - loss: 4.5806
Epoch 5/100
1841/1841 [==============================] - 8s 4ms/step - loss: 4.5178
Epoch 6/100
1841/1841 [==============================] - 8s 4ms/step - loss: 4.5134
Epoch 7/100
1841/1841 [==============================] - 8s 4ms/step - loss: 4.5341
Epoch 8/100
1841/1841 [==============================] - 8s 4ms/step - loss: 4.5548
Epoch 9/100
1841/1841 [==============================] - 8s 4ms/step - loss: 4.6232
Epoch 10/100
1841/1841 [==============================] - 8s 4ms/step - loss: 4.7057
Epoch 11/100
1841/1841 [==============================] - 8s 4ms/step - loss: 4.8611
Epoch 12/100
1841/1841 [==============================] - 8s 4ms/step - l

In [ ]:
def sample_with_temp(preds, temperature=1.0): 
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text
    seed_text = start_story + seed_text 

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0] 
        token_list = token_list[-max_sequence_len:] 
        token_list = np.reshape(token_list, (1, max_sequence_len))

        probs = model.predict(token_list, verbose=0)[0] 
        y_class = sample_with_temp(probs, temperature = temp) 

        output_word = tokenizer.index_word[y_class] if y_class > 0 else ''

        if output_word == "|": 
            break

        seed_text += output_word + '  ' 
        output_text += output_word + '  '

    return output_text

In [ ]:
seed_text = 'cat was eating'
next_words = 1
max_sequence_len = 6
temp = 0
seed_text = generate_text(seed_text, next_words, model, max_sequence_len, temp)

In [ ]:
while True:
  print(seed_text)
  seed_text = generate_text(seed_text, next_words, model, max_sequence_len, temp)

In [ ]:
print(seed_text)